In [15]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.nn as nn 
import os
from skimage import io, transform
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
from PIL import Image
import torch.nn.functional as F
from tensorboardX import SummaryWriter
import time

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
class ImageNetDataset(Dataset):
    
    def __init__(self, root_dir, data_folder, transform=None):
        self.data_path = os.path.join(root_dir,data_folder)
        self.data = os.listdir(self.data_path)
        self.root = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_name = self.data[index]
        img_path = os.path.join(self.data_path,img_name)
        image = io.imread(img_path)
        if self.transform:
            image = self.transform(image)
        
        return image,img_name

In [4]:
class addGaussian:
    
    def __init__(self, std_range):
        self.std_range = std_range

    def __call__(self,img):
        
        if len(np.shape(img))==2:
            img = np.stack((img,)*3, axis=-1)
    
        minval,maxval = self.std_range
        noise_img = img.astype(np.float)
        stddev = np.random.uniform(minval, maxval)
        noise = np.random.randn(*img.shape) * stddev
        noise_img += noise
        noise_img = np.clip(noise_img, 0, 255).astype(np.uint8)
        return noise_img

In [5]:
class Rescale(object):

    def __call__(self, image):

        if len(image.shape) == 2:
            image = np.stack((image,)*3, axis=-1)
        resize = transforms.Resize((256,256))

        tens = transforms.ToTensor()
        
        img = tens(resize(image))
        print(type(img))
        return img


In [6]:
num_epochs = 300
batch_size = 4 
learning_rate = 1e-3

In [7]:
train_dataset = ImageNetDataset('/home/turing/Documents/BE/','data/', 
                               transform=transforms.Compose([
                                   addGaussian((0,50)),
                                   transforms.ToPILImage(),
                                   transforms.Resize((256,256)),
                                   transforms.ToTensor(),

                               ]))

In [8]:
test_dataset = ImageNetDataset('/home/turing/Documents/BE/','data/', 
                               transform=transforms.Compose([
                                   addGaussian((0,50)),
                                   transforms.ToPILImage(),
                                   transforms.Resize((256,256)),
                                   transforms.ToTensor(),
                               ]))

In [9]:
i=Image.open('/home/turing/Documents/BE/data/ILSVRC2012_val_00005899.JPEG')
print(i.size)
i = np.stack((i,)*3, axis=-1)
print(i.shape)

(500, 500)
(500, 500, 3)


In [10]:
def getTrainLoader(batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True, num_workers=2)
    return(train_loader)
    

In [12]:
def getTestLoader(batch_size):
    test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=True, num_workers=2)
    
    return(test_loader)
    

In [13]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,48,3,padding=1)

        self.conv2 = nn.Conv2d(48,48,3,padding=1)

        self.pool = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(48,48,3,padding=1)

        self.conv4 = nn.Conv2d(48,48,3,padding=1)


        self.conv5 = nn.Conv2d(48,48,3,padding=1)

        self.conv6 = nn.Conv2d(48,48,3,padding=1)


        self.upscale = nn.Upsample(scale_factor=2)

        self.decov1 = nn.Conv2d(96,96,3,padding=1)


        self.decov2 = nn.Conv2d(96,96,3,padding=1)

        self.decov3 = nn.Conv2d(144,96,3,padding=1)


        self.decov4 = nn.Conv2d(96,96,3,padding=1)


        self.decov5 = nn.Conv2d(144,96,3,padding=1)

        self.decov6 = nn.Conv2d(96,96,3,padding=1)


        self.decov7 = nn.Conv2d(144,96,3,padding=1)

        self.decov8 = nn.Conv2d(96,96,3,padding=1)


        self.decov9 = nn.Conv2d(99,64,3,padding=1)

        self.decov10 = nn.Conv2d(64,32,3,padding=1)

        self.conv7 = nn.Conv2d(32,3,3,padding=1)
        
    def forward(self, x):
        
        skips = [x]
        
        # Enc. Conv0
        x = F.leaky_relu(self.conv1(x))
        
        # Enc. Conv1
        x = F.leaky_relu(self.conv2(x))
        
        #pool1
        x = self.pool(x)
        skips.append(x)
        
        # Enc. Conv2 and p00l2
        x = self.pool(F.leaky_relu(self.conv3(x)))
        
        skips.append(x)
        
        # Enc. Conv3 and p00l3
        x = self.pool(F.leaky_relu(self.conv4(x)))
        
        skips.append(x)
        
        # Enc. Conv4 and pool4
        x = self.pool(F.leaky_relu(self.conv5(x)))
        
        skips.append(x)
        
        # Enc. Conv5 and pool5
        x = self.pool(F.leaky_relu(self.conv6(x)))
        
        # Enc. Conv6
        x = F.leaky_relu(self.conv6(x))
        
        #---------#
        #upsample 5
        x = self.upscale(x)
        
        # concat 5
        x = torch.cat([x, skips.pop()],1)        
        
        x = F.leaky_relu(self.decov1(x))
        
        x = self.upscale(F.leaky_relu_(self.decov2(x)))

        x = torch.cat([x, skips.pop()],1)
        
        x = F.leaky_relu(self.decov3(x))
        x = self.upscale(F.leaky_relu(self.decov4(x)))
        
        # concat 3
        x = torch.cat([x, skips.pop()],1)
        x = F.leaky_relu(self.decov5(x))
        
        x = self.upscale(F.leaky_relu(self.decov6(x)))
        # concat2
        x = torch.cat([x, skips.pop()],1)
        x = F.leaky_relu(self.decov7(x))
        x = self.upscale(F.leaky_relu(self.decov8(x)))
        
        #concat 1
        x = torch.cat([x, skips.pop()],1)
        x = F.leaky_relu(self.decov9(x))
        x = F.leaky_relu(self.decov10(x))
        
        
        x = self.conv7(x)
        
        return x

In [14]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.MSELoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate, betas=(0.9,0.99),eaps=1e-8)
    
    return(loss, optimizer)

In [16]:
with SummaryWriter(comment='autoencoder') as w:
    model = Net
    w.add_graph(model, (dummy_input, ), verbose=True)



In [84]:
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    train_loader = getTrainLoader(batch_size)
    
    test_loader = getTestLoader(batch_size)
    
    n_batches = len(train_loader)
    
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    training_start_time = time.time()
    
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(zip(train_loader, test_loader)):

            optimizer.zero_grad()
            inputs, labels = data
#             print("input - ",inputs[0].shape)
            inputs = inputs[0].float()
            labels = labels[0].float()
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = net(inputs)
#             print("output = ", outputs.shape)
            
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            running_loss += loss_size.data[0]
            
            total_train_loss += loss_size.data[0]
            
            print(i,end='\r')
            
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                
                state = {
                    epoch = epoch,
                    i = i,
                    loss = loss,
                    optimizer = optimizer.state_dict(),
                    model = model.state_dict()
                }
                torch.save(state, 'checkpoint.pth')
                
                #Reset running loss and time
                
                writer = SummaryWriter('log')
                writer.add_scalar('loss', running_loss, i)
                
                for tag, value in net.named_parameters():
                    write.add_histogram(tag, value.data.cpu().numpy(), i+1)
                    
                
                running_loss = 0.0
                start_time = time.time
                
        print("finished Training")

In [43]:
net = Net().to(device)
# net = Net()

<bound method Module.parameters of Net(
  (conv1): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (upscale): Upsample(scale_factor=2, mode=nearest)
  (decov1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decov2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decov3): Conv2d(144, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decov4): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decov5): Conv2d(144, 96, kernel_size=(3, 3), s

In [ ]:
trainNet(net, batch_size=batch_size, n_epochs=num_epochs, learning_rate=learning_rate)

In [61]:
for tag, value in net.named_parameters():

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
